# ISCA2

Pathogenic variants in ISCA2 cause [Multiple mitochondrial dysfunctions syndrome 4 ](https://omim.org/entry/616370). This notebook contains information from several publications.

In [9]:
from pyphetools.creation import TemplateImporter
from pyphetools.visualization import IndividualTable, QcVisualizer
from IPython.display import display, HTML
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.66


In [10]:
template = "input/ISCA2_MMDS4_individuals.xlsx"
hp_json = "../hp.json"
created_by = "0000-0002-0736-9199"

In [13]:
timporter = TemplateImporter(template=template, hp_json=hp_json, created_by=created_by)
individual_list, cvalidator = timporter.import_phenopackets_from_template()

HPO version 2024-03-06
Created encoders for 77 fields
 we got OMIM:616370  NM_194279.4
We output 16 GA4GH phenopackets to the directory phenopackets


In [14]:
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,1


In [15]:
table = IndividualTable(cvalidator.get_error_free_individual_list())
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
29297947_P1 (MALE; P3Y),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.229G>A (homozygous),Increased CSF glycine concentration (HP:0500230); Hyperintensity of cerebral white matter on MRI (HP:0030890); Increased CSF lactate (HP:0002490); Wide nasal bridge (HP:0000431); Postnatal growth retardation (HP:0008897); Diffuse optic disc pallor (HP:0012512); Nasogastric tube feeding (HP:0040288); Hyperglycinuria (HP:0003108); Visual impairment (HP:0000505); Failure to thrive (HP:0001508); Hyperintensity of MRI T2 signal of the spinal cord (HP:0040272); Generalized hypotonia (HP:0001290); Leukodystrophy (HP:0002415); Low-set ears (HP:0000369); Increased CSF glutamate concentration (HP:0500200); Developmental regression (HP:0002376); Spasticity (HP:0001257); Global developmental delay (HP:0001263)
29297947_P2 (FEMALE; P9M),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.229G>A (homozygous),Hyperintensity of cerebral white matter on MRI (HP:0030890); Feeding difficulties in infancy (HP:0008872); Cutaneous syndactyly of toes (HP:0010621); Leg muscle stiffness (HP:0008969); Postnatal growth retardation (HP:0008897); Diffuse optic disc pallor (HP:0012512); Hypotonia (HP:0001252); Visual impairment (HP:0000505); Leukodystrophy (HP:0002415); Nystagmus (HP:0000639); Motor regression (HP:0033044); Short 4th metacarpal (HP:0010044); Spasticity (HP:0001257); Macrocephaly (HP:0000256); Global developmental delay (HP:0001263)
29359243_P1 (FEMALE; P3M),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.297del (heterozygous) NM_194279.3:c.334A>G (heterozygous),Abnormal diffusion weighted cerebral MRI morphology (HP:0032615); Increased CSF lactate (HP:0002490); Brisk reflexes (HP:0001348); Feeding difficulties (HP:0011968); Abnormality of the internal capsule (HP:0012502); Respiratory failure requiring assisted ventilation (HP:0004887); Nystagmus (HP:0000639); Motor regression (HP:0033044); Abnormal cerebral cortex morphology (HP:0002538); Severe muscular hypotonia (HP:0006829)
31279336_P1 (MALE; P7M),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.355G>A (homozygous),Insomnia (HP:0100785); Irritability (HP:0000737); Hypotonia (HP:0001252); Abnormal periventricular white matter morphology (HP:0002518); Malaise (HP:0033834); Abnormal cerebellum morphology (HP:0001317); Muscle stiffness (HP:0003552)
32424628_P1 (MALE; P11Y),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.5C>A (heterozygous) NM_194279.3:c.413C>G (heterozygous),Periventricular white matter hyperintensities (HP:0030891); Optic atrophy (HP:0000648); Abnormal pyramidal sign (HP:0007256); Vomiting (HP:0002013); Increased CSF lactate (HP:0002490); Arm dystonia (HP:0031960); Central nervous system cyst (HP:0030724); Thick corpus callosum (HP:0007074); Increased circulating lactate concentration (HP:0002151); Spastic tetraparesis (HP:0001285); Hyperreflexia (HP:0001347); Cognitive impairment (HP:0100543); Nystagmus (HP:0000639); Poor appetite (HP:0004396); Developmental regression (HP:0002376); Encephalopathy (HP:0001298)
25539947_P1 (FEMALE; P5Y),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.229G>A (homozygous),Periventricular white matter hyperintensities (HP:0030891); Optic atrophy (HP:0000648); Abnormal corpus callosum morphology (HP:0001273); Insomnia (HP:0100785); Irritability (HP:0000737); Hyperreflexia (HP:0001347); Abnormal midbrain morphology (HP:0002418); Developmental regression (HP:0002376); Spasticity (HP:0001257)
29122497_P1 (FEMALE; P1Y9M),Multiple mitochondrial dysfunctions syndrome 4 (OMIM:616370),NM_194279.3:c.229G>A (homozygous),Optic atrophy (HP:0000648); Lower limb spasticity (HP:0002061); Axial hypotonia (HP:0008936); Confluent hyperintensity of cerebral white matter on MRI (HP:0040330); Nystagmus (HP:0000639); Developmental regression (HP:0002376); Upper limb spasti